In [17]:
%pylab inline
pylab.rcParams['figure.figsize'] = 12,10

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [18]:
import os
import sys
from scipy.optimize import leastsq
from pylab import *
import csv
import mpld3

# General Functions

In [19]:
folderPath="/home/noza/work/Braindrop/CDB/BDTests/AHOsc_test/Measurements/"


# This is the readFile to use to take the average value of the sample of frequencies.
def readFileAvg(folderPath, fileName):
    data = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
    #print data
    sampleFreqs = data[:,1::2]
    #print sampleFreqs
    #print nanmean(sampleFreqs, axis=0)
    return nanmean(sampleFreqs, axis=0)

# This file will read the params csv file saved by the Parametric Analysis tool
def readParams(paramFile, paramTypes, displayVals=False, sortParams=True):
    returnVals = []
    for i, row in enumerate(csv.reader(paramFile)):
        if i<len(paramTypes):
            if paramTypes[i]=='int':
                tmpList = array([int(x) for x in array(row[1:])])
            elif paramTypes[i]=='double':
                tmpList = array(double(row[1:]))
            elif paramTypes[i]=='str':
                tmpList = array(row[1:])
            if sortParams:
                tmpList = array(sorted(tmpList))
            if displayVals:
                print "CurVars[",i,"]: ", tmpList
            
            returnVals.append(tmpList)
        else:
            print "Too many rows"
    return returnVals

def plotGen(Xs, Ys, temp, pltRange, plotType='.-', 
            plotLog=False, ttl='Default Title',
            xlbl='xLabel', ylbl='yLabel', color=(1,0,0,1)):
    rangeX, rangeY = pltRange[0], pltRange[1]    
    lenYs = len(Ys)
    if plotLog:
        loglog(Xs[:lenYs],Ys, plotType, label=str(temp)+"C", color=color)
    else:
        plot(Xs[:lenYs],Ys, plotType, label=str(temp)+"C", color=color)
    xlabel(xlbl)
    ylabel(ylbl)
    title(ttl)
    xlim(rangeX)
    ylim(rangeY)

# Plots a series of time points and their y-value at those times
def plotTimeSeries(xVals, yVals, clr=(1,0,0,1), 
                   ttl='', xLbl='', yLbl='',
                   xMin=0, xMax=10, yMin=0, yMax=10):
    if xMin==None:
        xMin = min(xVals)
    if xMax==None:
        xMax = max(xVals)
    if yMin==None:
        yMin = min(yVals)
    if yMax==None:
        yMax = max(yVals)
    plot(xVals,yVals, color=clr)
    xlabel(xLbl)
    ylabel(yLbl)
    title(ttl)
    xlim([xMin,xMax])
    ylim([yMin,yMax])

In [20]:
temps, = readParams(open(folderPath+"AH_InvChar/InvCharSweepParams.csv"), 
                                  ['int'], displayVals=True, sortParams=True)

CurVars[ 0 ]:  [ 0 20 25 30 50]


# Axon Hillock Delay Characterization
In this notebook, I am going to try and characterize the delay of the inverter and how it changes with temperature
## Inverter inversion points over temperature
The first thing I must do, in order to measure the delay is find out what the inversion voltage of the inverter is at various temperatures.  For posterities sake, I'm also eventually going to do this for thin-oxide devices.
### Thick-oxide

In [21]:
data = genfromtxt(folderPath+"AH_InvChar/AH_InvChar_Vinv1Leg.csv", 
                  dtype=float, delimiter=",", skip_header=1)
for i, curTemp in enumerate(temps):
    print curTemp,": ", data[::2][i]
invPoints = data[::2]

0 :  0.46953196228
20 :  0.470760659138
25 :  0.471012292507
30 :  0.471241586324
50 :  0.471930539161


## $V_{cap}$ plot over temperature

In [22]:
def timeAlignPlots(data, offsets, temps, tempsToPlot, yInterceptVal=0):
    for i, curTemp in enumerate(tempsToPlot):
        curBlue = 1.*(max(temps) - curTemp)/max(temps)
        curRed = 1.*curTemp/max(temps)
        curColor = (curRed, 0, curBlue, 1)
    #        curDchColor = (curRed, 0.5, curBlue, 1)

        #print "===============", curTemp
        curTimes = data[:,2*i]
        curVcapVal = data[:,2*i+1]

        # If offsets is a value greater than zero, use that value to offset the
        # waveforms equally
        if offsets[i]<=0:
            # Figure out where the system resets and subtract that value from the times
            timeMask = where(curTimes>.0*max(curTimes))[0]
            #print curTimes[timeMask[0]]
            mask = where(curVcapVal>yInterceptVal)[0]
            #print curTimes[mask[0]]
            mask = mask[mask>timeMask[0]]
            offsets[i] = curTimes[mask[0]]
        newVcapTimes = curTimes-offsets[i]

        # Plot the results
        subplot(len(tempsToPlot),1,i+1)
        plotTimeSeries(newVcapTimes, curVcapVal, clr=curColor,
                       ttl="$V_{cap}$\t$T$=%.0f C"%curTemp, xLbl='t (s)', yLbl='V (V)',
                       xMin=0, xMax=max(curTimes),
                       yMin=0.98*min(curVcapVal), yMax=1.02*max(curVcapVal))
        #vlines(curTimes[mask], 0, 1, color=curColor)
        #vlines(curTimes[timeMask[0]], 0, 1, color=(1,0,0,1))
    tight_layout()
    #print offsets
    return offsets

In [23]:
VcapLim = 0.475
tempsToPlot = temps[[0,2,4]]
offsets = zeros(len(tempsToPlot))

In [17]:
fileName = "AH_InvChar/AH_Vcap_IdealCS_C1_400x_C2_3x_Iin2x_10p_RBB-1V.csv"
VcapData = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
#offsets = timeAlignPlots(VcapData,offsets,temps,tempsToPlot,VcapLim)

## $V_{out}$ plot over temperature

In [18]:
fileName2 = "AH_InvChar/AH_Vout_IdealCS_C1_400x_C2_3x_Iin2x_10p_RBB-1V.csv"
VoutData = genfromtxt(folderPath+fileName2, dtype=float, delimiter=",", skip_header=1)
#offsets2 = timeAlignPlots(VoutData,offsets,temps,tempsToPlot)

## Determine error due to change in inverter delay

In [24]:
def getIntersectTimes(data, threshold):
    timeData, valData  = data[:,0], data[:,1]
    diffData = valData-threshold
    #print diffData
    diffDataSign = sign(diffData)
    #print diffDataSign
    changeData = ((np.roll(diffDataSign, 1) - diffDataSign) != 0).astype(int)
    changeData[0] = 0
    #print len(changeData), changeData
    indexOfChange = where(changeData==1)[0]
    #print indexOfChange
    dt = (timeData[indexOfChange]-timeData[indexOfChange-1])*1.
    dV = (valData[indexOfChange]-valData[indexOfChange-1])*1.
    intersectTimes = timeData[indexOfChange]-(dt/dV*(valData[indexOfChange]-threshold))
    return intersectTimes, indexOfChange

def getFreq(data, threshold):
    intersectTimes, indexOfChange = getIntersectTimes(data, threshold)
    freq1 = 1./diff(intersectTimes[::2])
    freq2 = 1./diff(intersectTimes[1::2])
    return freq1, freq2

def getPer(data, threshold):
    intersectTimes, indexOfChange = getIntersectTimes(data, threshold)
    per1 = diff(intersectTimes[::2])
    per2 = diff(intersectTimes[1::2])
    return per1, per2

def delayError(invPoints, VcapData, VoutData):
    avgInvDelays = []
    avgRegrPers = []
    avgPers = []
    avgFreqs = []
    for i, curInvPt in enumerate(invPoints):
        print invPoints[i]
        #print VcapData[:,2*i+1]
        curVcapData = VcapData[:,2*i:2*i+2]
        #print curVcapData
        curVcapData = curVcapData[~np.isnan(curVcapData).any(1)]
        #print curVcapData[:,1]

        curVoutData = VoutData[:,2*i:2*i+2]
        curVoutData = curVoutData[~np.isnan(curVoutData).any(1)]
        #print curVoutData[:,1]

        VcapIntersectTimes, VcapIntersectPts = getIntersectTimes(curVcapData, invPoints[i])
        #print "Vcap Intersect Times: ", VcapIntersectTimes
        VoutIntersectTimes, VoutIntersectPts = getIntersectTimes(curVoutData, invPoints[i])
        #print "Vout Intersect Times: ", VoutIntersectTimes
        invDelays = VoutIntersectTimes-VcapIntersectTimes
        #print "Vout - Vcap Delays: ", invDelays
        regrPer = VcapIntersectTimes[1:]-VoutIntersectTimes[:-1]
        #print "Regression Duration: ", regrPer
        stagePeriods = regrPer + invDelays[:-1]
        #print "Perc of half-period to rise: ", invDelays[:-1]/diff(VcapIntersectTimes)

        avgInvDelays.append(mean(invDelays))
        avgRegrPers.append(mean(regrPer))
        avgPers.append(2.*(mean(invDelays)+mean(regrPer)))
        avgFreqs.append(1./( 2.*(mean(invDelays)+mean(regrPer))))
        print "Avg Vo-Vc Delay: ", avgInvDelays[-1]
        print "Avg Regr Duration: ", avgRegrPers[-1]
        print "Avg Calc Period: ", avgPers[-1]
        print "Avg Calc Freq: ", avgFreqs[-1]
        
        #VcapF1, VcapF2 = getFreq(curVcapData,invPoints[i])
        #print "Vcap Freq", VcapF1, VcapF2
        #VoutF1, VoutF2 = getFreq(curVoutData,invPoints[i])
        #print "Vout Freq", VoutF1, VoutF2
        
        #VcapT1, VcapT2 = getPer(curVcapData,invPoints[i])
        #print "Vcap Periods: ", VcapT1, '\n', VcapT2
        #print "Avg Meas Period: ", mean([mean(VcapT1), mean(VcapT2)])
        #VoutT1, VoutT2 = getPer(curVoutData,invPoints[i])
        #print "Vout Periods: ", VoutT1, VoutT2
        
        #print "Vout Zipped Periods: ", array([x for t in zip(VoutT1, VoutT2) for x in t])
        #print "Vout Avg Period:", mean([mean(VoutT1), mean(VoutT2)])
        print
    return avgInvDelays, avgRegrPers, avgPers, avgFreqs

# Test Data to test some of the functions above        
#curVcapData = arange(0,10,1)
#curVcapData = array([concatenate([curVcapData,curVcapData+10,curVcapData+20,curVcapData+30]),
#                     5+concatenate([curVcapData,curVcapData[::-1],curVcapData,curVcapData[::-1]])]).T
##print curVcapData
#VcapIntersectTimes, VcapIntersectPts = getIntersectTimes(curVcapData, 8.1)
#f1, f2 = getFreq(curVcapData, 8.1)
##print VcapIntersectTimes, VcapIntersectPts
##print f1, f2

### Ideal Ckt

In [33]:
fileName = "AH_InvChar/AH_Vcap_IdealCkt.csv"
idealVcapData = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
#offsets = timeAlignPlots(idealVcapData,offsets,temps,tempsToPlot,VcapLim)
fileName2 = "AH_InvChar/AH_Vout_IdealCkt.csv"
idealVoutData = genfromtxt(folderPath+fileName2, dtype=float, delimiter=",", skip_header=1)
#offsets2 = timeAlignPlots(idealVoutData,offsets,temps,tempsToPlot)
#print idealVcapData[:,0:2]
avgInvDelays, avgRegrPers, avgPers, avgFreqs = delayError([.471], idealVcapData[:,0:2], idealVoutData[:,0:2])

0.471
Avg Vo-Vc Delay:  3.13099182142e-07
Avg Regr Duration:  2.17149577497e-05
Avg Calc Period:  4.40561138637e-05
Avg Calc Freq:  22698.3252107



### IdealCS C1 400x, C2 3x, Iin 2x 10p, RBB -1V

In [34]:
fileName = "AH_InvChar/AH_Vcap_IdealCS_C1_400x_C2_3x_Iin2x_10p_RBB-1V.csv"
VcapData = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
#offsets = timeAlignPlots(idealVcapData,offsets,temps,tempsToPlot,VcapLim)
fileName2 = "AH_InvChar/AH_Vout_IdealCS_C1_400x_C2_3x_Iin2x_10p_RBB-1V.csv"
VoutData = genfromtxt(folderPath+fileName2, dtype=float, delimiter=",", skip_header=1)
#offsets2 = timeAlignPlots(idealVoutData,offsets,temps,tempsToPlot)
avgInvDelays, avgRegrPers, avgPers, avgFreqs = delayError(invPoints, VcapData, VoutData)
#print '\n',avgInvDelays,'\n', avgRegrPers,'\n', avgPers,'\n', avgFreqs

print [avgInvDelays[i]/avgRegrPers[i] for i in range(len(avgRegrPers))]

print (max(avgInvDelays)-min(avgInvDelays))/mean(avgInvDelays[2])
print (max(avgRegrPers)-min(avgRegrPers))/mean(avgRegrPers[2])
print (max(avgPers)-min(avgPers))/mean(avgPers[2])
print (max(avgFreqs)-min(avgFreqs))/mean(avgFreqs[2])

0.46953196228
Avg Vo-Vc Delay:  7.93419901974e-07
Avg Regr Duration:  1.88043288537e-05
Avg Calc Period:  3.91954975113e-05
Avg Calc Freq:  25513.1345051

0.470760659138
Avg Vo-Vc Delay:  3.78665416719e-07
Avg Regr Duration:  1.80139901067e-05
Avg Calc Period:  3.67853110468e-05
Avg Calc Freq:  27184.7640143

0.471012292507
Avg Vo-Vc Delay:  3.19190257176e-07
Avg Regr Duration:  1.78640198023e-05
Avg Calc Period:  3.63664201189e-05
Avg Calc Freq:  27497.8949462

0.471241586324
Avg Vo-Vc Delay:  2.7075875002e-07
Avg Regr Duration:  1.77383199909e-05
Avg Calc Period:  3.60181574818e-05
Avg Calc Freq:  27763.7744381

0.471930539161
Avg Vo-Vc Delay:  1.47035964634e-07
Avg Regr Duration:  1.73090206307e-05
Avg Calc Period:  3.49121131906e-05
Avg Calc Freq:  28643.3535129

[0.042193470883692664, 0.021020629770315619, 0.017867773362813227, 0.015264058273754516, 0.0084947593380016208]
2.02507414562
0.0837050249349
0.117784052064
0.1138348595


### IdealCS C1 1x, C2 3x, Iin 2x 10p, RBB -1V

In [35]:
fileName = "AH_InvChar/AH_Vcap_IdealCS_C1_1x_C2_3x_Iin2x_10p_RBB-1V.csv"
VcapData = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
#offsets = timeAlignPlots(idealVcapData,offsets,temps,tempsToPlot,VcapLim)
fileName2 = "AH_InvChar/AH_Vout_IdealCS_C1_1x_C2_3x_Iin2x_10p_RBB-1V.csv"
VoutData = genfromtxt(folderPath+fileName2, dtype=float, delimiter=",", skip_header=1)
#offsets2 = timeAlignPlots(idealVoutData,offsets,temps,tempsToPlot)
avgInvDelays, avgRegrPers, avgPers, avgFreqs = delayError(invPoints, VcapData, VoutData)
#print '\n',avgInvDelays,'\n', avgRegrPers,'\n', avgPers,'\n', avgFreqs

print [avgInvDelays[i]/avgRegrPers[i] for i in range(len(avgRegrPers))]

print (max(avgInvDelays)-min(avgInvDelays))/mean(avgInvDelays[2])
print (max(avgRegrPers)-min(avgRegrPers))/mean(avgRegrPers[2])
print (max(avgPers)-min(avgPers))/mean(avgPers[2])
print (max(avgFreqs)-min(avgFreqs))/mean(avgFreqs[2])

0.46953196228
Avg Vo-Vc Delay:  0.0
Avg Regr Duration:  1.86594560417e-05
Avg Calc Period:  3.73189120835e-05
Avg Calc Freq:  26796.065163

0.470760659138
Avg Vo-Vc Delay:  0.0
Avg Regr Duration:  1.85015620404e-05
Avg Calc Period:  3.70031240808e-05
Avg Calc Freq:  27024.745203

0.471012292507
Avg Vo-Vc Delay:  0.0
Avg Regr Duration:  1.84722049407e-05
Avg Calc Period:  3.69444098813e-05
Avg Calc Freq:  27067.6944959

0.471241586324
Avg Vo-Vc Delay:  0.0
Avg Regr Duration:  1.84456594375e-05
Avg Calc Period:  3.68913188749e-05
Avg Calc Freq:  27106.6481356

0.471930539161
Avg Vo-Vc Delay:  0.0
Avg Regr Duration:  1.83564096359e-05
Avg Calc Period:  3.67128192718e-05
Avg Calc Freq:  27238.4420438

[0.0, 0.0, 0.0, 0.0, 0.0]
nan
0.0164055350634
0.0164055350634
0.0163433528087


/opt/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in double_scalars


### IdealCS C1 400x, C2 4x, Iin 2x 10p, RBB -1V, Ideal Inverter1
In this test, I made the first inverter ideal to see if the variation in the delay to temperature continues to propogate through the circuit if the first inverter is ideal.

In [36]:
fileName = "AH_InvChar/AH_Vcap_IdealCS_IdealInv1_C1_400x_C2_4x_Iin2x_10p_RBB-1V.csv"
VcapData = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
#offsets = timeAlignPlots(idealVcapData,offsets,temps,tempsToPlot,VcapLim)
fileName2 = "AH_InvChar/AH_Vout_IdealCS_IdealInv1_C1_400x_C2_4x_Iin2x_10p_RBB-1V.csv"
VoutData = genfromtxt(folderPath+fileName2, dtype=float, delimiter=",", skip_header=1)
#offsets2 = timeAlignPlots(idealVoutData,offsets,temps,tempsToPlot)
avgInvDelays, avgRegrPers, avgPers, avgFreqs = delayError(invPoints, VcapData, VoutData)
#print '\n',avgInvDelays,'\n', avgRegrPers,'\n', avgPers,'\n', avgFreqs

print [avgInvDelays[i]/avgRegrPers[i] for i in range(len(avgRegrPers))]

print (max(avgInvDelays)-min(avgInvDelays))/mean(avgInvDelays[2])
print (max(avgRegrPers)-min(avgRegrPers))/mean(avgRegrPers[2])
print (max(avgPers)-min(avgPers))/mean(avgPers[2])
print (max(avgFreqs)-min(avgFreqs))/mean(avgFreqs[2])

0.46953196228
Avg Vo-Vc Delay:  4.26199369056e-08
Avg Regr Duration:  2.34176917774e-05
Avg Calc Period:  4.69206234287e-05
Avg Calc Freq:  21312.5897937

0.470760659138
Avg Vo-Vc Delay:  2.06923178464e-08
Avg Regr Duration:  2.31798088064e-05
Avg Calc Period:  4.64010022485e-05
Avg Calc Freq:  21551.2586268

0.471012292507
Avg Vo-Vc Delay:  1.75702838145e-08
Avg Regr Duration:  2.312819006e-05
Avg Calc Period:  4.62915206877e-05
Avg Calc Freq:  21602.2283378

0.471241586324
Avg Vo-Vc Delay:  1.49964637621e-08
Avg Regr Duration:  2.3078352233e-05
Avg Calc Period:  4.61866973934e-05
Avg Calc Freq:  21651.2558038

0.471930539161
Avg Vo-Vc Delay:  8.46099174822e-09
Avg Regr Duration:  2.28877298785e-05
Avg Calc Period:  4.57923817405e-05
Avg Calc Freq:  21837.6935637

[0.0018199888063565547, 0.00089268716663096204, 0.00075969125854044508, 0.00064980652044371401, 0.00036967369822782852]
1.94413166675
0.022914110337
0.0243725345682
0.0243078520311


### IdealCS C1 400x, C2 4x, Iin 2x 10p, RBB -1V, Ideal Inverters

In [37]:
fileName = "AH_InvChar/AH_Vcap_IdealCS_IdealInvs_C1_400x_C2_4x_Iin2x_10p_RBB-1V.csv"
VcapData = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
#offsets = timeAlignPlots(idealVcapData,offsets,temps,tempsToPlot,VcapLim)
fileName2 = "AH_InvChar/AH_Vout_IdealCS_IdealInvs_C1_400x_C2_4x_Iin2x_10p_RBB-1V.csv"
VoutData = genfromtxt(folderPath+fileName2, dtype=float, delimiter=",", skip_header=1)
#offsets2 = timeAlignPlots(idealVoutData,offsets,temps,tempsToPlot)
avgInvDelays, avgRegrPers, avgPers, avgFreqs = delayError(invPoints, VcapData, VoutData)
#print '\n',avgInvDelays,'\n', avgRegrPers,'\n', avgPers,'\n', avgFreqs

print [avgInvDelays[i]/avgRegrPers[i] for i in range(len(avgRegrPers))]

print (max(avgInvDelays)-min(avgInvDelays))/mean(avgInvDelays[2])
print (max(avgRegrPers)-min(avgRegrPers))/mean(avgRegrPers[2])
print (max(avgPers)-min(avgPers))/mean(avgPers[2])
print (max(avgFreqs)-min(avgFreqs))/mean(avgFreqs[2])

0.46953196228
Avg Vo-Vc Delay:  3.04388817509e-07
Avg Regr Duration:  2.44825347116e-05
Avg Calc Period:  4.95738470582e-05
Avg Calc Freq:  20171.9265165

0.470760659138
Avg Vo-Vc Delay:  3.00766798969e-07
Avg Regr Duration:  2.44974987352e-05
Avg Calc Period:  4.95965310683e-05
Avg Calc Freq:  20162.7004643

0.471012292507
Avg Vo-Vc Delay:  3.0564011339e-07
Avg Regr Duration:  2.44965782558e-05
Avg Calc Period:  4.96044367383e-05
Avg Calc Freq:  20159.487049

0.471241586324
Avg Vo-Vc Delay:  3.04937035454e-07
Avg Regr Duration:  2.4500359254e-05
Avg Calc Period:  4.9610592579e-05
Avg Calc Freq:  20156.9855955

0.471930539161
Avg Vo-Vc Delay:  3.0935108637e-07
Avg Regr Duration:  2.45165786132e-05
Avg Calc Period:  4.96518593991e-05
Avg Calc Freq:  20140.232654

[0.012432896393076563, 0.012277449311055491, 0.012476849223528473, 0.012446227105981807, 0.012618036604963556]
0.0280862590502
0.00138974109976
0.00157268877669
0.00157215620138


### IdealCS C1 400x, C2 3x, Iin 2x 10p, RBB -1V, Ideal Inverters Ainv 800x
Previous to this test, the gain of the inverter (Ainv) was set to 80 to match roughly what the gain of the inverters are in real life.

This test will hopefully show that by increasing the gain of the inverters, I am able to  decrease the Vo-Vc delay, and therefore, minimize the dependence on a large capacitor.

In [38]:
fileName = "AH_InvChar/AH_Vcap_IdealCS_IdealInvs_C1_400x_C2_3x_Iin2x_10p_RBB-1V_Ainv800.csv"
VcapData = genfromtxt(folderPath+fileName, dtype=float, delimiter=",", skip_header=1)
#offsets = timeAlignPlots(idealVcapData,offsets,temps,tempsToPlot,VcapLim)
fileName2 = "AH_InvChar/AH_Vout_IdealCS_IdealInvs_C1_400x_C2_3x_Iin2x_10p_RBB-1V_Ainv800.csv"
VoutData = genfromtxt(folderPath+fileName2, dtype=float, delimiter=",", skip_header=1)
#offsets2 = timeAlignPlots(idealVoutData,offsets,temps,tempsToPlot)
avgInvDelays, avgRegrPers, avgPers, avgFreqs = delayError(invPoints, VcapData, VoutData)
#print '\n',avgInvDelays,'\n', avgRegrPers,'\n', avgPers,'\n', avgFreqs

print [avgInvDelays[i]/avgRegrPers[i] for i in range(len(avgRegrPers))]

print (max(avgInvDelays)-min(avgInvDelays))/mean(avgInvDelays[2])
print (max(avgRegrPers)-min(avgRegrPers))/mean(avgRegrPers[2])
print (max(avgPers)-min(avgPers))/mean(avgPers[2])
print (max(avgFreqs)-min(avgFreqs))/mean(avgFreqs[2])

0.46953196228
Avg Vo-Vc Delay:  3.2603029359e-09
Avg Regr Duration:  1.89394771528e-05
Avg Calc Period:  3.78854749115e-05
Avg Calc Freq:  26395.3402283

0.470760659138
Avg Vo-Vc Delay:  3.56367980754e-09
Avg Regr Duration:  1.89505849247e-05
Avg Calc Period:  3.7908297209e-05
Avg Calc Freq:  26379.4491873

0.471012292507
Avg Vo-Vc Delay:  5.71740396043e-09
Avg Regr Duration:  1.89510493071e-05
Avg Calc Period:  3.79135334221e-05
Avg Calc Freq:  26375.8059389

0.471241586324
Avg Vo-Vc Delay:  3.8069365604e-09
Avg Regr Duration:  1.89563280156e-05
Avg Calc Period:  3.79202699044e-05
Avg Calc Freq:  26371.1203143

0.471930539161
Avg Vo-Vc Delay:  3.37129661442e-09
Avg Regr Duration:  1.89688211513e-05
Avg Calc Period:  3.79443848958e-05
Avg Calc Freq:  26354.3605397

[0.0001721432386748282, 0.00018805117740203344, 0.00030169326604420095, 0.00020082668738717404, 0.00017772831466606004]
0.429758163239
0.00154841022025
0.00155379831165
0.00155368479255


## Function to calculate percentage error given jump distance, at various temperatures

In [23]:
def percErrorDV(Vinv1, V1, Vinv2, V2):
    dV1 = abs(V1 - Vinv1)
    dV2 = abs(V2 - Vinv2)
    #print dV1, dV2
    #print mean((dV1, dV2))
    #print abs(dV1-dV2)
    
    return abs(dV1-dV2)/mean((dV1,dV2))

print "Inversion Voltages: ", invPoints
print "3W Inverter"
print percErrorDV(invPoints[0], 724.3731e-3, invPoints[4], 714.5288e-3)
print percErrorDV(invPoints[0], 252.48606e-3, invPoints[4], 237.27334e-3)

print "1W Inverter, with C1 150a, C2 150a"
print percErrorDV(invPoints[0], 613.8756e-3, invPoints[4], 628.0861e-3)
print percErrorDV(invPoints[0], 348.6124e-3, invPoints[4], 330.5263e-3)

Inversion Voltages:  [ 0.46953196  0.47076066  0.47101229  0.47124159  0.47193054]
3W Inverter
0.0492235915234
0.0779773121992
1W Inverter
0.0786153385795
0.156178560761
